In [1]:
from database.connect import getConnection
import JsonHandler as jh
import mysql
import pandas as pd

/Users/alicialarsen/anaconda3/envs/ok/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

In [3]:
print(jh.languages_list)


['ar', 'en', 'fr', 'pt', 'es', 'it', 'de', 'hi']


In [4]:
airlines_dict = {'KLM': ['klm'],
                'AirFrance':['airfrance',
                            'air france'],
                'British_Airways': ['british_airways',
                                    'british airways'],
                'AmericanAir': ['americanair',
                                'american airlines'],
                'Lufthansa': ['lufthansa'],
                'AirBerlin': ['airberlin',
                                'air berlin'],
                'AirBerlin assist': ['airberlin assist',
                                    'air berlin assist',
                                    'airberlinassist'],
                'easyJet': ['easyjet'],
                'RyanAir': ['ryanair'],
                'SingaporeAir': ['singaporeair',
                                'singapore airlines'],
                'Qantas': ['qantas'],
                'EtihadAirways': ['etihad airways',
                                'etihadairways',
                                'etihad'],
                'VirginAtlantic': ['virgin atlantic',
                                    'virginatlantic'],
            }

In [5]:
# Create a cursor to execute SQL queries
cursor = connection.cursor()

for key, value in airlines_dict.items():
    query = f"""
        SELECT
            t.language AS Language,
            (SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) / COUNT(*)) * 100 AS ParticipatedPercentage,
            COUNT(*) AS TotalConversations,
            SUM(CASE WHEN c.Airline LIKE '%{key}%' THEN 1 ELSE 0 END) AS ParticipatedConversations
        FROM
            conversations c
            INNER JOIN part_of p ON c.id = p.cID
            INNER JOIN tweets t ON t.id = p.tID
        WHERE
            (t.airlines LIKE '%lazy_query_strat_ignore_this%'
    """
    for airline in value:
        query += f" OR t.airlines LIKE '%{airline}%'"

    query += """
        )
        GROUP BY Language
    """

    cursor.execute(query)
    result = cursor.fetchall()
    print(key)
    print(result)





ProgrammingError: 1146 (42S02): Table 'dbl.conversations' doesn't exist

In [11]:
# create the dataframe to store the language counts
df = pd.DataFrame(columns=['language', 'sentiment'])

# get all the sentiment scores per language
for language in jh.languages_list:
    query = """
    SELECT sentiment
    FROM tweets
    WHERE language = %s
    """
    cursor.execute(query, (language,))
    sentiments = cursor.fetchall()
    sentiment_list = [sentiment[0] for sentiment in sentiments]

    # add the language and the sentiment to the dataframe
    df.loc[len(df.index)] = [language, sentiment_list]

df.head()

,language,sentiment
0,ar,"[-0.0234151, -0.0694676, -0.00520112, -0.35635..."
1,en,"[-0.308782, 0.549322, 0.864803, -0.792114, 0.2..."
2,fr,"[-0.873595, -0.873595, -0.873595, -0.873595, -..."
3,pt,"[0.554107, 0.127242, 0.0215067, -0.3548, 0.032..."
4,sp,[]


In [12]:
# Close the cursor and connection
cursor.close()
connection.close()
